## Import packages

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from utils.tools import forward_fill_pipeline, normalize_dataframe, normalize_df_with_statistics, calculate_missing_rate, export_missing_mask_pipeline

data_dir = "./tjh/"
Path(os.path.join(data_dir, 'processed_sample')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(data_dir, 'statistics')).mkdir(parents=True, exist_ok=True)

SEED = 42

## Read data from files

### Record feature names

In [ ]:
basic_records = ['PatientID', 'RecordTime', 'AdmissionTime', 'DischargeTime']
target_features = ['Outcome', 'LOS']
demographic_features = ['Sex', 'Age']
labtest_features = ['Hypersensitive cardiac troponinI', 'hemoglobin', 'Serum chloride', 'Prothrombin time', 'procalcitonin', 'eosinophils(%)', 'Interleukin 2 receptor', 'Alkaline phosphatase', 'albumin', 'basophil(%)', 'Interleukin 10', 'Total bilirubin', 'Platelet count', 'monocytes(%)', 'antithrombin', 'Interleukin 8', 'indirect bilirubin', 'Red blood cell distribution width ', 'neutrophils(%)', 'total protein', 'Quantification of Treponema pallidum antibodies', 'Prothrombin activity', 'HBsAg', 'mean corpuscular volume', 'hematocrit', 'White blood cell count', 'Tumor necrosis factorα', 'mean corpuscular hemoglobin concentration', 'fibrinogen', 'Interleukin 1β', 'Urea', 'lymphocyte count', 'PH value', 'Red blood cell count', 'Eosinophil count', 'Corrected calcium', 'Serum potassium', 'glucose', 'neutrophils count', 'Direct bilirubin', 'Mean platelet volume', 'ferritin', 'RBC distribution width SD', 'Thrombin time', '(%)lymphocyte', 'HCV antibody quantification', 'D-D dimer', 'Total cholesterol', 'aspartate aminotransferase', 'Uric acid', 'HCO3-', 'calcium', 'Amino-terminal brain natriuretic peptide precursor(NT-proBNP)', 'Lactate dehydrogenase', 'platelet large cell ratio ', 'Interleukin 6', 'Fibrin degradation products', 'monocytes count', 'PLT distribution width', 'globulin', 'γ-glutamyl transpeptidase', 'International standard ratio', 'basophil count(#)', 'mean corpuscular hemoglobin ', 'Activation of partial thromboplastin time', 'Hypersensitive c-reactive protein', 'HIV antibody quantification', 'serum sodium', 'thrombocytocrit', 'ESR', 'glutamic-pyruvic transaminase', 'eGFR', 'creatinine']
require_impute_features = labtest_features
normalize_features = ['Age'] + labtest_features + ['LOS']

drop_columns = labtest_features

In [ ]:
df = pd.read_csv(os.path.join(data_dir, "processed_sample", f"tjh_dataset_formatted.csv"))
# df

## Stratified split dataset into `Training`, `Validation` and `Test` sets

- Stratified dataset according to `Outcome` column
- Baseline: 70% Training, 10% Validation, 20% Test (No Calibration)
    - Name: train, val, test


In [ ]:
# Group the dataframe by patient ID
grouped = df.groupby('PatientID')

# Get the patient IDs and outcomes
patients = np.array(list(grouped.groups.keys()))
patients_outcome = np.array([grouped.get_group(patient_id)['Outcome'].iloc[0] for patient_id in patients])

# Get the train_val/test patient IDs
train_val_patients, test_patients = train_test_split(patients, test_size=20/100, random_state=SEED, stratify=patients_outcome)

# Get the train/val patient IDs
train_val_patients_outcome = np.array([grouped.get_group(patient_id)['Outcome'].iloc[0] for patient_id in train_val_patients])
train_patients, val_patients = train_test_split(train_val_patients, test_size=10/80, random_state=SEED, stratify=train_val_patients_outcome)


In [ ]:
# Create train, val, test, [traincal, calib] dataframes for the current fold
train_df = df[df['PatientID'].isin(train_patients)]
val_df = df[df['PatientID'].isin(val_patients)]
test_df = df[df['PatientID'].isin(test_patients)]


In [ ]:
save_dir = os.path.join(data_dir, 'processed_sample', 'fold_0') # forward fill
Path(save_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
# Calculate the mean and std of the train set (include age, lab test features, and LOS) on the data in 5% to 95% quantile range
train_df, val_df, test_df, default_fill, los_info, train_mean, train_std = normalize_dataframe(train_df, val_df, test_df, normalize_features)

train_x, train_y, train_pid = forward_fill_pipeline(train_df, default_fill, demographic_features, labtest_features, target_features, require_impute_features)
val_x, val_y, val_pid = forward_fill_pipeline(val_df, default_fill, demographic_features, labtest_features, target_features, require_impute_features)
test_x, test_y, test_pid = forward_fill_pipeline(test_df, default_fill, demographic_features, labtest_features, target_features, require_impute_features)

# Save the imputed dataset to pickle file
pd.to_pickle(train_x, os.path.join(save_dir, "train_x.pkl"))
pd.to_pickle(train_y, os.path.join(save_dir, "train_y.pkl"))
pd.to_pickle(train_pid, os.path.join(save_dir, "train_pid.pkl"))
pd.to_pickle(val_x, os.path.join(save_dir, "val_x.pkl"))
pd.to_pickle(val_y, os.path.join(save_dir, "val_y.pkl"))
pd.to_pickle(val_pid, os.path.join(save_dir, "val_pid.pkl"))
pd.to_pickle(test_x, os.path.join(save_dir, "test_x.pkl"))
pd.to_pickle(test_y, os.path.join(save_dir, "test_y.pkl"))
pd.to_pickle(test_pid, os.path.join(save_dir, "test_pid.pkl"))
pd.to_pickle(los_info, os.path.join(save_dir, "los_info.pkl")) # LOS statistics (calculated from the train set)

In [ ]:
"""
for fold_1 to fold_9, reduce sample size from 10% to 90%
"""
save_dir = os.path.join(data_dir, 'processed_sample', 'fold_0') # forward fill
# Load the data
train_x = pd.read_pickle(os.path.join(save_dir, "train_x.pkl"))
train_y = pd.read_pickle(os.path.join(save_dir, "train_y.pkl"))

train_x_ = []
train_y_ = []
for i in range(len(train_x)):
    for j in range(len(train_x[i])):
        cur_x = train_x[i][j]
        cur_y = train_y[i][j]
        train_x_.append((i, cur_x))
        train_y_.append((i, cur_y))

for fold in range(1, 10):

    print("Fold: {}".format(fold))

    # Set the percentage to be dropped
    drop_percentage = 0.1*fold

    np.random.seed(42)
    # Randomly select the indices to be dropped
    drop_indices = np.random.choice(np.arange(len(train_x_)), int(len(train_x_) * drop_percentage))

    # Drop the selected indices from the lists
    fold_train_x_ = [train_x_[i] for i in range(len(train_x_)) if i not in drop_indices]
    fold_train_y_ = [train_y_[i] for i in range(len(train_y_)) if i not in drop_indices]    

    pid_ = []
    x_ = []
    y_ = []
    for i in range(len(fold_train_x_)):
        cur_pid = fold_train_x_[i][0]
        cur_x = fold_train_x_[i][1]
        cur_y = fold_train_y_[i][1]

        if cur_pid not in pid_:
            pid_.append(cur_pid)
            x_.append([cur_x])
            y_.append([cur_y])
        else:
            x_[pid_.index(cur_pid)].append(cur_x)
            y_[pid_.index(cur_pid)].append(cur_y)

    save_dir = os.path.join(data_dir, 'processed_sample', f'fold_{fold}') # forward fill
    Path(save_dir).mkdir(parents=True, exist_ok=True)

    pd.to_pickle(x_, os.path.join(save_dir, "train_x.pkl"))
    pd.to_pickle(y_, os.path.join(save_dir, "train_y.pkl"))
    pd.to_pickle(pid_, os.path.join(save_dir, "train_pid.pkl"))